In [1]:
import pandas as pd
import numpy as np
import annoy
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.applications import vgg16
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import keras
from keras.layers import Dense, Dropout, Flatten
from keras import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import GlobalMaxPooling2D
from PIL import Image
import os
from sklearn.metrics.pairwise import cosine_similarity
import cv2
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer
from keras.applications.xception import Xception,preprocess_input
from keras.preprocessing import image
from keras.layers import Input
from keras.backend import reshape
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.models import Sequential,load_model,Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,Reshape,UpSampling2D,Input,Lambda,Conv2DTranspose


In [2]:
images=pd.read_csv('E:/H and M/HM Personal_Laptop/img_pres.csv')
images=images.drop('Unnamed: 0',axis=1)
images.rename(columns={'0':'img_id'},inplace=True)
print(images.shape)
images.head()

(105100, 1)


,img_id
0,0108775015.jpg
1,0108775044.jpg
2,0108775051.jpg
3,0110065001.jpg
4,0110065002.jpg


In [3]:
images['path']=images['img_id'].apply(lambda x: f"E:/H and M/HM Personal_Laptop/images/{x[0:3]}/{x}")

In [4]:
images.iloc[0,1]

'E:/H and M/HM Personal_Laptop/images/010/0108775015.jpg'

In [16]:
images.to_csv('image_info.csv')

# Image similarity model

In [5]:
imgs_model_width, imgs_model_height = 224, 224
nb_closest_images = 5 

In [6]:
vgg_model = vgg16.VGG16(weights='imagenet')

# remove the last layers in order to get features instead of predictions
feat_extractor = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("fc2").output)
top_model = feat_extractor.output
top_model = Dense(4096, activation='relu')(top_model)
top_model = Dense(1072, activation='relu')(top_model)
output_layer=Dense(600,activation='relu')(top_model)
final_model=Model(inputs=vgg_model.input,outputs=output_layer)
# print the layers of the CNN
final_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [8]:
importedImages = []

In [9]:
for f in tqdm(images['path'][0:5000]):
    filename =f
    original = load_img(filename, target_size=(224, 224))
    numpy_image = img_to_array(original)
    image_batch = np.expand_dims(numpy_image, axis=0)
    importedImages.append(image_batch)
    
#processed_imgs = preprocess_input(images.copy())

  0%|          | 0/5000 [00:00<?, ?it/s]

In [10]:
imagess = np.vstack(importedImages)
imagess.shape

(5000, 224, 224, 3)

In [11]:
processed_imgs=preprocess_input(imagess)

In [12]:
imgs_features = feat_extractor.predict(processed_imgs)

print("features successfully extracted!")
imgs_features.shape

157/157 [==============================] - 836s 5s/step
features successfully extracted!


(5000, 4096)

In [ ]:
imgs_features

In [13]:
cosSimilarities = cosine_similarity(imgs_features)
cos_similarities_df = pd.DataFrame(cosSimilarities, columns=images['path'][0:5000], index=images['path'][0:5000])
cos_similarities_df.head()

path,E:/H and M/HM Personal_Laptop/images/010/0108775015.jpg,E:/H and M/HM Personal_Laptop/images/010/0108775044.jpg,E:/H and M/HM Personal_Laptop/images/010/0108775051.jpg,E:/H and M/HM Personal_Laptop/images/011/0110065001.jpg,E:/H and M/HM Personal_Laptop/images/011/0110065002.jpg,E:/H and M/HM Personal_Laptop/images/011/0110065011.jpg,E:/H and M/HM Personal_Laptop/images/011/0111565001.jpg,E:/H and M/HM Personal_Laptop/images/011/0111565003.jpg,E:/H and M/HM Personal_Laptop/images/011/0111586001.jpg,E:/H and M/HM Personal_Laptop/images/011/0111593001.jpg,...,E:/H and M/HM Personal_Laptop/images/048/0489217007.jpg,E:/H and M/HM Personal_Laptop/images/048/0489435001.jpg,E:/H and M/HM Personal_Laptop/images/048/0489435002.jpg,E:/H and M/HM Personal_Laptop/images/048/0489435009.jpg,E:/H and M/HM Personal_Laptop/images/048/0489435010.jpg,E:/H and M/HM Personal_Laptop/images/048/0489435011.jpg,E:/H and M/HM Personal_Laptop/images/048/0489435012.jpg,E:/H and M/HM Personal_Laptop/images/048/0489435014.jpg,E:/H and M/HM Personal_Laptop/images/048/0489435015.jpg,E:/H and M/HM Personal_Laptop/images/048/0489435016.jpg
path,,,,,,,,,,,,,,,,,,,,,
E:/H and M/HM Personal_Laptop/images/010/0108775015.jpg,1.000000,0.961681,0.974386,0.977737,0.961188,0.972273,0.941894,0.957899,0.956268,0.958372,...,0.911624,0.951709,0.951384,0.954769,0.954861,0.950637,0.955817,0.947313,0.954170,0.945386
E:/H and M/HM Personal_Laptop/images/010/0108775044.jpg,0.961681,1.000000,0.978197,0.958795,0.993788,0.981855,0.962285,0.958465,0.963825,0.964833,...,0.918834,0.980947,0.973577,0.976915,0.974529,0.977722,0.975748,0.953639,0.976721,0.961853
E:/H and M/HM Personal_Laptop/images/010/0108775051.jpg,0.974386,0.978197,1.000000,0.965839,0.975869,0.977835,0.948924,0.966358,0.962569,0.964599,...,0.924103,0.970994,0.963249,0.966562,0.967037,0.969613,0.966992,0.957639,0.967525,0.961195
E:/H and M/HM Personal_Laptop/images/011/0110065001.jpg,0.977737,0.958795,0.965839,1.000000,0.966312,0.983325,0.943709,0.943825,0.946089,0.949469,...,0.913624,0.958085,0.963144,0.962034,0.963686,0.957867,0.963485,0.949646,0.962482,0.949798
E:/H and M/HM Personal_Laptop/images/011/0110065002.jpg,0.961188,0.993788,0.975869,0.966312,1.000000,0.988004,0.962125,0.951366,0.954847,0.958377,...,0.913356,0.986874,0.980306,0.981372,0.980496,0.983150,0.981786,0.955436,0.981588,0.965257


In [14]:
cos_similarities_df.to_csv('similarities_image.csv')